In [8]:
%pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("DISASTERS.csv")

In [11]:
print(df.head())

   Year   Seq Glide Disaster Group Disaster Subgroup      Disaster Type  \
0  1900  9002   NaN        Natural    Climatological            Drought   
1  1900  9001   NaN        Natural    Climatological            Drought   
2  1902    12   NaN        Natural       Geophysical         Earthquake   
3  1902     3   NaN        Natural       Geophysical  Volcanic activity   
4  1902    10   NaN        Natural       Geophysical  Volcanic activity   

  Disaster Subtype Disaster Subsubtype   Event Name     Country  ISO  \
0          Drought                 NaN          NaN  Cabo Verde  CPV   
1          Drought                 NaN          NaN       India  IND   
2  Ground movement                 NaN          NaN   Guatemala  GTM   
3         Ash fall                 NaN  Santa Maria   Guatemala  GTM   
4         Ash fall                 NaN  Santa Maria   Guatemala  GTM   

            Region Continent                   Location Origin  \
0   Western Africa    Africa                Countr

In [12]:
# Supression des colonnes inutiles car elles ne vont pas nous servire dans notre etude
print("\n Suppression des colonnes administratives...")

# Colonnes à supprimer (peu utiles pour ML)
columns_to_drop = ['Seq', 'Glide', 'OFDA Response', 'Appeal', 'Declaration','Disaster Subsubtype'
                   'Aid Contribution','ISO',  'Associated Dis', 'Associated Dis2',
                   'Event Name', 'Origin','Location', 'Local Time', 'River Basin',
                   'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Dis Mag Scale', 'Geo Locations']

df_clean = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
print(f" Colonnes supprimées: {[col for col in columns_to_drop if col in df.columns]}")


 Suppression des colonnes administratives...
 Colonnes supprimées: ['Seq', 'Glide', 'OFDA Response', 'Appeal', 'Declaration', 'ISO', 'Associated Dis', 'Associated Dis2', 'Event Name', 'Origin', 'Location', 'Local Time', 'River Basin', 'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Dis Mag Scale', 'Geo Locations']


In [13]:
# selection des colonnes importantes
print("\n Sélection des colonnes pertinentes...")

columns_to_keep = [
    'Year', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
    'Disaster Subtype', 'Country', 'Region', 'Continent',
     'Latitude', 'Longitude',
    'Start Year', 'Start Month', 'Start Day',
    'End Year', 'End Month', 'End Day',
    'Dis Mag Value',
    'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
    'Total Affected', 'Total Damages (\'000 US$)', 'Insured Damages (\'000 US$)'
]

columns_to_keep = [col for col in columns_to_keep if col in df_clean.columns]
df_clean = df_clean[columns_to_keep].copy()

print(f"{len(columns_to_keep)} colonnes sélectionnées")


 Sélection des colonnes pertinentes...
24 colonnes sélectionnées


In [14]:
# nettoyage des valeurs manquantes

print("\n Nettoyage des valeurs manquantes...")

# Supprimer les lignes avec trop de valeurs manquantes (>50%)
df_clean = df_clean.dropna(thresh=len(df_clean.columns) * 0.5)

# Remplir les valeurs manquantes pour les colonnes d'impact par 0
impact_cols = ['Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
               'Total Affected', 'Total Damages (\'000 US$)', 'Insured Damages (\'000 US$)']

for col in impact_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0).clip(lower=0)

# Coordonnées: Nettoyer et convertir en numérique, puis remplir par la médiane du pays
if 'Latitude' in df_clean.columns:
    # Remove non-numeric characters and convert to float
    df_clean['Latitude'] = df_clean['Latitude'].astype(str).str.replace(r'[^\d.-]', '', regex=True)
    df_clean['Latitude'] = pd.to_numeric(df_clean['Latitude'], errors='coerce')
    df_clean['Latitude'] = df_clean.groupby('Country')['Latitude'].transform(
        lambda x: x.fillna(x.median() if not x.isnull().all() else np.nan)
    )
if 'Longitude' in df_clean.columns:
    # Remove non-numeric characters and convert to float
    df_clean['Longitude'] = df_clean['Longitude'].astype(str).str.replace(r'[^\d.-]', '', regex=True)
    df_clean['Longitude'] = pd.to_numeric(df_clean['Longitude'], errors='coerce')
    df_clean['Longitude'] = df_clean.groupby('Country')['Longitude'].transform(
        lambda x: x.fillna(x.median() if not x.isnull().all() else np.nan)
    )
if 'Latitude' in df_clean.columns: df_clean['Latitude'] = df_clean['Latitude'].fillna(df_clean['Latitude'].median())
if 'Longitude' in df_clean.columns: df_clean['Longitude'] = df_clean['Longitude'].fillna(df_clean['Longitude'].median())

print(f" Lignes restantes: {len(df_clean)}")


 Nettoyage des valeurs manquantes...


 Lignes restantes: 15818


In [15]:
print("\n Création des colonnes de dates...")

def create_date(row, date_type='start'):
    try:
        if date_type == 'start':
            year = row['Start Year'] if pd.notna(row['Start Year']) else row['End Year']
            month = row['Start Month'] if pd.notna(row['Start Month']) else 1
            day = row['Start Day'] if pd.notna(row['Start Day']) else 1
        else:
            year = row['End Year'] if pd.notna(row['End Year']) else row['Start Year']
            month = row['End Month'] if pd.notna(row['End Month']) else 12
            day = row['End Day'] if pd.notna(row['End Day']) else 31

        return pd.to_datetime(f"{int(year)}-{int(month)}-{int(day)}", errors='coerce')
    except:
        return pd.NaT

# Création des dates
df_clean['Start_Date'] = df_clean.apply(lambda row: create_date(row, 'start'), axis=1)
df_clean['End_Date']   = df_clean.apply(lambda row: create_date(row, 'end'), axis=1)

# Remplissage pour cohérence : si une date manque, on prend l'autre
df_clean.loc[df_clean['Start_Date'].isna(), 'Start_Date'] = \
    df_clean.loc[df_clean['Start_Date'].isna(), 'End_Date']

df_clean.loc[df_clean['End_Date'].isna(), 'End_Date'] = \
    df_clean.loc[df_clean['End_Date'].isna(), 'Start_Date']

# Durée en jours
df_clean['Duration_Days'] = (df_clean['End_Date'] - df_clean['Start_Date']).dt.days
df_clean['Duration_Days'] = df_clean['Duration_Days'].fillna(1).clip(lower=0)

print(" Dates et durée créées")



 Création des colonnes de dates...
 Dates et durée créées


In [16]:
#Création des features temporelles
print("\n Création des features temporelles...")

df_clean['Month'] = df_clean['Start_Date'].dt.month
df_clean['Season'] = df_clean['Month'].apply(lambda x:
    'Winter' if x in [12, 1, 2] else
    'Spring' if x in [3, 4, 5] else
    'Summer' if x in [6, 7, 8] else
    'Fall' if pd.notna(x) and x in [9, 10, 11] else 'Unknown'
)

print(" Features temporelles créées")


 Création des features temporelles...
 Features temporelles créées


In [17]:
# traitement des valeurs manquantes pour Dis Mag Value

if 'Dis Mag Value' in df_clean.columns:
    # Remplacer les valeurs manquantes par la médiane
    df_clean['Dis Mag Value'] = df_clean['Dis Mag Value'].fillna(df_clean['Dis Mag Value'].median())


In [18]:
print("\n Encodage des variables catégorielles...")

categorical_cols = [
    'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
    'Disaster Subtype', 'Country', 'Region', 'Continent',
    'Season', 'Magnitude_Category'
]

# Dictionnaire qui va stocker les légendes de toutes les colonnes
legendes = {}

for col in categorical_cols:
    if col in df_clean.columns:

        #  créer une liste des valeurs uniques
        valeurs_uniques = df_clean[col].astype(str).unique()

        #  créer le dictionnaire {valeur : numéro}
        mapping = {val: i for i, val in enumerate(valeurs_uniques)}

        #  enregistrer la légende
        legendes[col] = mapping

        #  remplacer dans le dataframe
        df_clean[col] = df_clean[col].astype(str).map(mapping)

print("Encodage terminé.\n")

#  Affichage de la légende
for col, mapping in legendes.items():
    print(f"\nLégende pour {col}:")
    for val, code in mapping.items():
        print(f"  {code} → {val}")



 Encodage des variables catégorielles...


Encodage terminé.


Légende pour Disaster Group:
  0 → Natural

Légende pour Disaster Subgroup:
  0 → Geophysical
  1 → Meteorological
  2 → Hydrological
  3 → Climatological
  4 → Biological
  5 → Extra-terrestrial

Légende pour Disaster Type:
  0 → Earthquake
  1 → Volcanic activity
  2 → Mass movement (dry)
  3 → Storm
  4 → Flood
  5 → Landslide
  6 → Wildfire
  7 → Epidemic
  8 → Drought
  9 → Extreme temperature 
  10 → Fog
  11 → Insect infestation
  12 → Impact
  13 → Animal accident
  14 → Glacial lake outburst

Légende pour Disaster Subtype:
  0 → Ground movement
  1 → Ash fall
  2 → Rockfall
  3 → Tropical cyclone
  4 → nan
  5 → Landslide
  6 → Avalanche
  7 → Forest fire
  8 → Convective storm
  9 → Riverine flood
  10 → Viral disease
  11 → Drought
  12 → Mudslide
  13 → Tsunami
  14 → Heat wave
  15 → Land fire (Brush, Bush, Pasture)
  16 → Bacterial disease
  17 → Coastal flood
  18 → Cold wave
  19 → Flash flood
  20 → Parasitic disease
  21 → Locust
  22 → Severe wint

In [19]:
# Supprimer les colonnes de dates individuelles (on garde Start_Date, End_Date)
cols_to_drop = ['Start Year', 'Start Month', 'Start Day',
                'End Year', 'End Month', 'End Day']
df_clean = df_clean.drop(columns=[col for col in cols_to_drop if col in df_clean.columns])

In [ ]:
# Sauvegarde du fichier nettoyé
print("\n Sauvegarde du fichier nettoyé...")

output_filename = 'DISASTERS_CLEANED.csv'
df_clean.to_csv(output_filename, index=False)

print(f" Fichier sauvegardé: {output_filename}")


 Sauvegarde du fichier nettoyé...
 Fichier sauvegardé: disasters_cleaned_2.csv


In [21]:
# Rapport final
print("\n" + "="*60)
print(" RAPPORT DE PRÉTRAITEMENT")
print("="*60)

print(f"\n Dataset original: {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f" Dataset nettoyé: {df_clean.shape[0]} lignes, {df_clean.shape[1]} colonnes")
print(f" Lignes supprimées: {df.shape[0] - df_clean.shape[0]}")

print("\n Nouvelles colonnes créées:")
new_cols = ['Start_Date', 'End_Date', 'Duration_Days', 'Month', 'Season']
for col in new_cols:
    if col in df_clean.columns:
        print(f"  ✓ {col}")



print("\n Top 5 pays les plus touchés:")
print(df_clean['Country'].value_counts().head(5))

print("\n Types de catastrophes:")
print(df_clean['Disaster Type'].value_counts().head(5))


print("PRÉTRAITEMENT TERMINÉ AVEC SUCCÈS!")




 RAPPORT DE PRÉTRAITEMENT

 Dataset original: 16126 lignes, 45 colonnes
 Dataset nettoyé: 15818 lignes, 23 colonnes
 Lignes supprimées: 308

 Nouvelles colonnes créées:
  ✓ Start_Date
  ✓ End_Date
  ✓ Duration_Days
  ✓ Month
  ✓ Season

 Top 5 pays les plus touchés:
Country
66    1070
8      968
3      740
74     667
13     565
Name: count, dtype: int64

 Types de catastrophes:
Disaster Type
4    5492
3    4472
0    1544
7    1478
5     762
Name: count, dtype: int64
PRÉTRAITEMENT TERMINÉ AVEC SUCCÈS!
